In [ ]:
import os
from urllib.parse import urlparse
from model_registry import ModelRegistry

In [ ]:
def infer_model_format(uri_or_repo):
    ext = os.path.splitext(uri_or_repo)[1].lower()
    return {
        ".onnx": "onnx",
        ".pkl": "sklearn",
        ".pt": "pytorch",
        ".joblib": "sklearn",
        ".h5": "keras",
        ".sav": "sklearn"
    }.get(ext, "unknown")


In [ ]:
def is_hf_model(uri_or_repo):
    return "/" in uri_or_repo and not uri_or_repo.startswith("http")


In [ ]:
def register_model_kubeflow(registry_url: str, author: str, model_uri_or_repo: str, model_name: str,version: str = "1.0.0",
    description: str = "Auto-registered model", model_path: str = None,model_format_name: str = None, model_format_version: str = "1", 
    storage_key: str = None, metadata: dict = None         
):
    registry = ModelRegistry(registry_url, author=author)

    # Check whether this is a Hugging Face model
    huggingface = is_hf_model(model_uri_or_repo)

    # Auto-infer model format if not provided
    if not model_format_name:
        model_format_name = infer_model_format(model_path or model_uri_or_repo)

    # Auto-infer model path from URI if not provided
    if not model_path:
        parsed = urlparse(model_uri_or_repo)
        model_path = parsed.path.lstrip("/") if parsed.scheme in ["http", "https"] else model_uri_or_repo

    if huggingface:
        model = registry.register_hf_model(
            repo_id=model_uri_or_repo,
            path=model_path,
            version=version,
            model_name=model_name,
            description=description,
            model_format_name=model_format_name,
            model_format_version=model_format_version,
            metadata=metadata or {}
        )
    # Register generic URI-based model
    else:
        model = registry.register_model(
            model_name=model_name,
            uri=model_uri_or_repo,
            version=version,
            description=description,
            model_format_name=model_format_name,
            model_format_version=model_format_version,
            storage_key=storage_key,
            storage_path=model_path,
            metadata=metadata or {}
        )
    print(" Model registered:", model)
    print(" Registered model:", registry.get_registered_model(model_name))
    print(" Model version:", registry.get_model_version(model_name, version))
    print(" Artifact info:", registry.get_model_artifact(model_name, version))

    return model

In [ ]:
if __name__ == "__main__":
    register_model_kubeflow(
        registry_url="http://localhost:8080",
        author="thirumalesh",
        model_uri_or_repo="https://storage.my-org.com/models/road-segmentation.onnx",
        model_name="road-segmentation-v2",
        version="2.0.1",
        description="Road segmentation model trained on Indian highway dataset",
        storage_key="s3-roadmodels",
        metadata={
            "accuracy": 0.937,
            "precision": 0.91,
            "recall": 0.92,
            "framework": "onnx",
            "task": "segmentation",
            "dataset": "IndianHighway-v3",
            "training_time": "3h45m",
            "epochs": 25,
            "learning_rate": 0.001,
            "optimizer": "Adam",
            "batch_size": 32,
            "loss_function": "cross_entropy",
            "inference_latency_ms": 12.4,
            "developer": "cv-team",
            "tags": ["production", "segmentation", "cv"]
        }
    )
